In [15]:
import minio
import os
import cv2 as cv
import matplotlib.pyplot as plt

minio_client = minio.Minio(
    os.environ['MINIO_ENDPOINT'],
    access_key=os.environ['MINIO_ACCESS_KEY'],
    secret_key=os.environ['MINIO_SECRET_KEY'],
    secure=False
)

In [58]:
class MultiCam:
    def __init__(self, video_list):
        self.video_list = video_list
        self.cap_list: list[cv.VideoCapture] = []
        for video in video_list:
            self.cap_list.append(cv.VideoCapture(video))
        self.frame_count = [int(cap.get(cv.CAP_PROP_FRAME_COUNT)) for cap in self.cap_list]

    def seek(self, frame, sync=True):
        max_frame = max(self.frame_count)
        if frame > max_frame:
            print('Frame number exceeds the max frame count')
            return

        if sync:
            for idx, cap in enumerate(self.cap_list):
                frame_diff = max_frame - self.frame_count[idx]
                if frame_diff > 0:
                    frame_diff -= 1
                cap.set(cv.CAP_PROP_POS_FRAMES, frame - frame_diff)

        else:            
            for cap in self.cap_list:                
                if frame > self.frame_count[self.cap_list.index(cap)]:
                    print('Frame number exceeds the total frame count')
                    return
                cap.set(cv.CAP_PROP_POS_FRAMES, frame)

    def get_frame(self, frame):
        self.seek(frame)
        frame_list = []
        for cap in self.cap_list:
            ret, frame = cap.read()
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            frame_list.append(frame)
        return frame_list

    def release(self):
        for cap in self.cap_list:
            cap.release()

def plot_frame(frame_list, height=10, landscape=False):
    # Plot 3 images side by side without axis and any gap between them
    if landscape:
        fig, ax = plt.subplots(1, len(frame_list),figsize=(height*1920/1080*3.1, height))
    else:
        fig, ax = plt.subplots(len(frame_list), 1, figsize=(height*1920/1080, height*3))
    for idx, frame in enumerate(frame_list):
        ax[idx].imshow(frame)
        ax[idx].axis('off')
        ax[idx].set_aspect('auto')
        ax[idx].set_xticks([])
        ax[idx].set_yticks([])
    fig.tight_layout()
    plt.show()

In [65]:
bucket = 'highjump'
path = 'raw_data/2025/03/14/run14/'
download_path = 'data/videos/'

video_list = list(minio_client.list_objects(bucket, path))

for video in video_list:
    minio_client.fget_object(bucket, video.object_name, download_path + video.object_name.split('/')[-1])

In [ ]:
multi_cam = MultiCam([download_path + 'sidecam.mp4', download_path + 'curve_cam.mp4', download_path + 'centercam.mp4'])

print(multi_cam.frame_count)
frame_list_1 = multi_cam.get_frame(3865)
plot_frame(frame_list_1, 12)